In [ ]:
#| echo: false
#| output: false
Pkg.activate(@__DIR__)
Pkg.instantiate()

## Overview

This tutorial will give some examples of using `Turing.jl` and Markov Chain Monte Carlo to sample from posterior distributions.

## Setup


In [ ]:
using Turing
using Distributions
using Plots
default(fmt = :png) # the tide gauge data is long, this keeps images a manageable size
using LaTeXStrings
using StatsPlots
using Measures
using StatsBase
using Optim
using Random
using DataFrames
using DataFramesMeta
using Dates
using CSV

As this tutorial involves random number generation, we will set a random seed to ensure reproducibility.


In [ ]:
Random.seed!(1);

## Fitting A Linear Regression Model

Let's start with a simple example: fitting a linear regression model to simulated data.

::: {.callout-tip}
## Positive Control Tests

Simulating data with a known data-generating process and then trying to obtain the parameters for that process is an important step in any workflow.
:::

### Simulating Data

The data-generating process for this example will be:
$$
\begin{gather}
y = 5 + 2x + \varepsilon \\
\varepsilon \sim \text{Normal}(0, 3),
\end{gather}
$$
where $\varepsilon$ is so-called "white noise", which adds stochasticity to the data set. The generated dataset is shown in @fig-scatter-regression.


In [ ]:
#| column: margin
#| label: fig-scatter-regression
#| fig-cap: Scatterplot of our generated data.
#| echo: false

# create trend for data
x = rand(Uniform(0, 20), 20)
y = 5 .+ 2 * x
# sample and add noise
ε = rand(Normal(0, 5), 20)
y .+= ε

scatter(x, y, label="Data", xlabel=L"$x$", ylabel=L"$y$", markershape=:o, markersize=10)
plot!(tickfontsize=16, guidefontsize=18, legendfontsize=16) # adjust fonts and marker sizes for the margin plot

### Model Specification

The statistical model for a standard linear regression problem is
$$
\begin{gather}
y = a + bx + \varepsilon \\
\varepsilon \sim \text{Normal}(0, \sigma).
\end{gather}
$$

Rearranging, we can rewrite the likelihood function as:
$$y \sim \text{Normal}(\mu, \sigma),$$
where $\mu = a + bx$. This means that we have three parameters to fit: $a$, $b$, and $\sigma^2$.

Next, we need to select priors on our parameters. We'll use relatively generic distributions to avoid using the information we have (since we generated the data ourselves), but in practice, we'd want to use any relevant information that we had from our knowledge of the problem. Let's use relatively diffuse normal distributions for the trend parameters $a$ and $b$ and a half-normal distribution (a normal distribution truncated at 0, to only allow positive values) for the variance $\sigma^2$, as recommended by @Gelman2006-wv.

$$
\begin{gather}
a \sim \text{Normal(0, 10)} \\
b \sim \text{Normal(0, 10)} \\
\sigma \sim \text{Half-Normal}(0, 25)
\end{gather}
$$

### Using Turing

#### Coding the Model

`Turing.jl` uses the `@model` macro to specify the model function. We'll follow the setup in the [Turing documentation](https://turinglang.org/dev/tutorials/05-linear-regression).

To specify distributions on parameters (and the data, which can be thought of as uncertain parameters in Bayesian statistics), use a tilde `~`, and use equals `=` for transformations (which we don't have in this case).


In [ ]:
@model function linear_regression(x, y)
    # set priors
    σ ~ truncated(Normal(0, 25); lower=0)      # <1>
    a ~ Normal(0, 10)                           # <2>
    b ~ Normal(0, 10)                           # <2>

    # compute the likelihood
    for i = 1:length(y)                         # <3>
        # compute the mean value for the data point
        μ = a + b * x[i]
        y[i] ~ Normal(μ, σ)
    end
end

1. Standard deviations must be positive, so we use a normal distribution truncated at zero.
2. We'll keep these both relative uninformative to reflect a more "realistic" modeling scenario.
3. In this case, we specify the likelihood with a loop. We could also rewrite this as a joint likelihood over all of the data using linear algebra, which might be more efficient for large and/or complex models or datasets, but the loop is more readable in this simple case.


#### Fitting The Model

Now we can call the sampler to draw from the posterior. We'll use the [No-U-Turn sampler](https://en.wikipedia.org/wiki/Hamiltonian_Monte_Carlo#No_U-Turn_Sampler) [@Hoffman2014-gv], which is a Hamiltonian Monte Carlo algorithm (a different category of MCMC sampler than the Metropolis-Hastings algorithm discussed in class). We'll also use 4 chains so we can test that the chains are well-mixed, and each chain will be run for 5,000 iterations^[Hamiltonian Monte Carlo samplers often need to be run for fewer iterations than Metropolis-Hastings samplers, as the exploratory step uses information about the gradient of the statistical model, versus the random walk of Metropolis-Hastings. The disadvantage is that this gradient information must be available, which is not always the case for external simulation models. Simulation models coded in Julia can usually be automatically differentiated by Turing's tools, however.]


In [ ]:
# set up the sampler
model = linear_regression(x, y)     # <1>
n_chains = 4                         # <2>
n_per_chain = 5000                  # <3>
chain = sample(model, NUTS(), MCMCThreads(), n_per_chain, n_chains, drop_warmup=true) # <4>
@show chain                         # <5>

1. Initialize the model with the data.
2. We use multiple chains to help diagnose convergence.
3. This sets the number of iterations for each chain.
4. Sample from the posterior using NUTS and drop the iterations used to warmup the sampler. The `MCMCThreads()` call tells the sampler to use available processor threads for the multiple chains, but it will just sample them in serial if only one thread exists.
5. The `@show` macro makes the display of the output a bit cleaner.

How can we interpret the output? The first parts of the summary statistics are straightforward: we get the mean, standard deviation, and Monte Carlo standard error (`mcse`) of each parameter. We also get information about the effective sample size (ESS)^[The ESS reflects the efficiency of the sampler: this is an estimate of the equivalent number of independent samples; the more correlated the samples, the lower the ESS.] and $\hat{R}$, which measures the ratio of within-chain variance and across-chain variance as a check for convergence^[The closer $\hat{R}$ is to 1, the better.].

In this case, we can see that we were generally able to recover the "true" data-generating values of $\sigma = 4$ and $b = 2$, but $a$ is slightly off (the mean is 3, rather than the data-generating value of 5). In fact, there is substantial uncertainty about $a$, with a 95% credible interval of $(3.1, 11.4)$  (compared to $(1.4, 2.2)$ for $b$). This isn't surprising: given the variance of the noise $\sigma^2$, there are many different intercepts which could fit within that spread.

Let's now plot the chains for visual inspection.


In [ ]:
#| label: fig-chains-regression
#| fig-cap: 'Output from the MCMC sampler. Each row corresponds to a different parameter: $\sigma$, $a$, and $b$. Each chain is shown in a different color. The left column shows the sampler traceplots, and the right column the resulting posterior distributions.'

plot(chain)

We can see from @fig-chains-regression that our chains mixed well and seem to have converged to similar distributions! The traceplots have a "hairy caterpiller" appearance, suggesting relatively little autocorrelation. We can also see how much more uncertainty there is with the intercept $a$, while the slope $b$ is much more constrained.

Another interesting comparison we can make is with the maximum-likelihood estimate (MLE), which we can obtain through optimization.


In [ ]:
#| column: margin

mle_model = linear_regression(x, y)
mle = optimize(mle_model, MLE()) # <1>
coef(mle)

1. This is where we use the `Optim.jl` package in this tutorial.

We could also get the maximum *a posteriori* (MAP) estimate, which includes the prior density, by replacing `MLE()` with `MAP()`.

### Model Diagnostics and Posterior Predictive Checks

One advantage of the Bayesian modeling approach here is that we have access to a *generative model*, or a model which we can use to generate datasets. This means that we can now use Monte Carlo simulation, sampling from our posteriors, to look at how uncertainty in the parameter estimates propagates through the model. Let's write a function which gets samples from the MCMC chains and generates datasets.


In [ ]:
function mc_predict_regression(x, chain)
    # get the posterior samples
    a = Array(group(chain, :a))     # <1>
    b = Array(group(chain, :b))     # <1>
    σ = Array(group(chain, :σ))   # <1>

    # loop and generate alternative realizations
    μ = a' .+ x * b'
    y = zeros((length(x), length(a)))
    for i = 1:length(a)
        y[:, i] = rand.(Normal.(μ[:, i], σ[i]))
    end
    return y
end

1. The `Array(group())` syntax is more general than we need, but is useful if we have multiple variables which were sampled as a group, for example multiple regression coefficients. Otherwise, we can just use *e.g.* `Array(chain, :a)`.

Now we can generate a predictive interval and median and compare to the data.


In [ ]:
x_pred = 0:20
y_pred = mc_predict_regression(x_pred, chain)

Notice the dimension of `y_pred`: we have 20,000 columns, because we have 4 chains with 5,000 samples each. If we had wanted to subsample (which might be necessary if we had hundreds of thousands or millions of samples), we could have done that within `mc_linear_regression` before simulation.


In [ ]:
#| output: false

# get the boundaries for the 95% prediction interval and the median
y_ci_low = quantile.(eachrow(y_pred), 0.025)
y_ci_hi = quantile.(eachrow(y_pred), 0.975)
y_med = quantile.(eachrow(y_pred), 0.5)

Now, let's plot the prediction interval and median, and compare to the original data.


In [ ]:
#| label: fig-prediction-regression
#| fig-cap: Posterior 95% predictive interval and median for the linear regression model. The data is plotted in red for comparison.

# plot prediction interval
plot(x_pred, y_ci_low, fillrange=y_ci_hi, xlabel=L"$x$", ylabel=L"$y$", fillalpha=0.3, fillcolor=:blue, label="95% Prediction Interval", legend=:topleft, linealpha=0) # <1>
plot!(x_pred, y_med, color=:blue, label="Prediction Median") # <2>
scatter!(x, y, color=:red, label="Data") # <3>

1. Plot the 95% posterior prediction interval as a shaded blue ribbon.
2. Plot the posterior prediction median as a blue line.
3. Plot the data as discrete red points.

From @fig-prediction-regression, it looks like our model might be slightly under-confident, as with 20 data points, we would expect 5% of them (or 1 data point) to be outside the 95% prediction interval. It's hard to tell with only 20 data points, though! We could resolve this by tightening our priors, but this depends on how much information we used to specify them in the first place. The goal shouldn't be to hit a specific level of uncertainty, but if there is a sound reason to tighten the priors, we could do so.

Now let's look at the residuals from the posterior median and the data. The partial autocorrelations plotted in @fig-residuals-regression are not fully convincing, as there are large autocorrelation coefficients with long lags, but the dataset is quite small, so it's hard to draw strong conclusions. We won't go further down this rabbit hole as we know our data-generating process involved independent noise, but for a real dataset, we might want to try a model specification with autocorrelated errors to compare.


In [ ]:
#| column: margin
#| label: fig-residuals-regression
#| fig-cap: 'Partial autocorrelation function of model residuals, relative to the predictive median.'

# calculate the median predictions and residuals
y_pred_data = mc_predict_regression(x, chain)
y_med_data = quantile.(eachrow(y_pred_data), 0.5)
residuals = y_med_data .- y

# plot the residuals and a line to show the zero
plot(pacf(residuals, 1:4), line=:stem, marker=:circle, legend=:false, grid=:false, linewidth=2, xlabel="Lag", ylabel="Partial Autocorrelation", markersize=8, tickfontsize=14, guidefontsize=16, legendfontsize=16)
hline!([0], linestyle=:dot, color=:red) # <1>

## Fitting Extreme Value Models to Tide Gauge Data

Let's now look at an example of fitting an extreme value distribution (namely, a [generalized extreme value distribution](https://en.wikipedia.org/wiki/Generalized_extreme_value_distribution), or GEV) to tide gauge data. GEV distributions have three parameters:

- $\mu$, the *location* parameter, which reflects the positioning of the bulk of the GEV distribution;
- $\sigma$, the *scale* parameter, which reflects the width of the bulk;
- $\xi$, the *shape* parameter, which reflects the thickness and boundedness of the tail.

The shape parameter $\xi$ is often of interest, as there are three classes of GEV distributions corresponding to different signs:

- $\xi < 0$ means that the distribution is bounded;
- $\xi = 0$ means that the distribution has a thinner tail, so the "extreme extremes" are less likely;
- $\xi > 0$ means that the distribution has a thicker tail.

### Load Data

First, let's load the data. We'll use [data from the University of Hawaii Sea Level Center](https://uhslc.soest.hawaii.edu/datainfo/) [@Caldwell2015-kc] for San Francisco, from 1897-2013. If you don't have this data and are working with the notebook, download it [here](https://uhslc.soest.hawaii.edu/data/csv/rqds/hourly/h551a.csv). We'll assume it's in a `data/` subdirectory, but change the path as needed.

The dataset consists of dates and hours and the tide-gauge measurement, in mm. We'll load the dataset into a `DataFrame`.


In [ ]:
function load_data(fname)
    date_format = DateFormat("yyyy-mm-dd HH:MM:SS")
    df = @chain fname begin                        # <1> 
        CSV.File(; delim=',', header=false)        # <2>
        DataFrame                                  # <3>
        rename("Column1" => "year",                # <4>
                "Column2" => "month",              # <4>
                "Column3" => "day",                # <4>
                "Column4" => "hour",               # <4>
                "Column5" => "gauge")              # <4>
        # need to reformat the decimal date in the data file
        @transform :datetime = DateTime.(:year, :month, :day, :hour) # <5>
        # replace -99999 with missing
        @transform :gauge = ifelse.(abs.(:gauge) .>= 9999, missing, :gauge) # <6>
        select(:datetime, :gauge) # <7>
    end
    return df
end

1. This uses the `DataFramesMeta.jl` package, which makes it easy to string together commands to load and process data
2. Load the file, assuming there is no header.
3. Convert to a `DataFrame`.
4. Rename columns for ease of access.
5. Reformat the decimal datetime provided in the file into a Julia `DateTime`.
6. Replace missing data with `missing`.
7. Select only the `:datetime` and `:gauge` columns.


In [ ]:
#| label: tbl-data
#| tbl-cap: 'Processed hourly tide gauge data from San Francisco, from 8/1/1897-1/31/2023.'
#| column: margin

dat = load_data("data/h551a.csv")
first(dat, 6)

In [ ]:
#| label: fig-raw-data
#| fig-cap: Hourly mean water at the San Francisco tide gauge from 1897-2023.
#| fig-alt: Tide gauge data

@df dat plot(:datetime, :gauge, label="Observations", bottom_margin=9mm) # <1>
xaxis!("Date", xrot=30)
yaxis!("Mean Water Level")

1. This uses the `DataFrame` plotting recipe with the `@df` macro from `StatsPlots.jl`. This is not needed (you could replace *e.g.* `:datetime` with `dat.datetime`), but it cleans things up slightly.

Next, we need to detrend the data to remove the impacts of sea-level rise. We do this by removing a one-year moving average, centered on the data point, per the recommendation of @Arns2013-tl.


In [ ]:
#| label: fig-data-detrend
#| fig-cap: 'Mean water level from the San Francisco tide gauge, detrended using a 1-year moving average centered on the data point, per the recommendation of @Arns2013-tl.'
#| fig-alt: Detrended tide gauge data

# calculate the moving average and subtract it off
ma_length = 366
ma_offset = Int(floor(ma_length/2))
moving_average(series,n) = [mean(@view series[i-n:i+n]) for i in n+1:length(series)-n]
dat_ma = DataFrame(datetime=dat.datetime[ma_offset+1:end-ma_offset], residual=dat.gauge[ma_offset+1:end-ma_offset] .- moving_average(dat.gauge, ma_offset))

# plot
@df dat_ma plot(:datetime, :residual, label="Detrended Observations", bottom_margin=9mm)
xaxis!("Date", xrot=30)
yaxis!("Mean Water Level")

The last step in preparing the data is to find the annual maxima. We can do this using the `groupby`, `transform`, and `combine` functions from `DataFrames.jl`, as below.


In [ ]:
#| label: fig-annmax-histogram
#| fig-cap: Histogram of annual block maxima  from 1898-2022 from the San Francisco tide gauge dataset.
#| fig-alt: Annual maxima histogram

# calculate the annual maxima
dat_ma = dropmissing(dat_ma) # <1>
dat_annmax = combine(dat_ma -> dat_ma[argmax(dat_ma.residual), :],      # <2>
                groupby(transform(dat_ma, :datetime => x->year.(x)), :datetime_function)) # <2>
delete!(dat_annmax, nrow(dat_annmax)) # <3>

# make a histogram of the maxima to see the distribution
histogram(dat_annmax.residual, label=false)
ylabel!("Count")
xlabel!("Mean Water Level (mm)")

1. If we don't drop the values which are missing, they will affect the next call to `argmax`.
2. This first groups the data based on the year (with `groupby` and using `Dates.year()` to get the year of each data point), then pulls the rows which correspond to the maxima for each year (using `argmax`).
3. This will delete the last year, in this case 2023, because the dataset only goes until March 2023 and this data point is almost certainly an outlier due to the limited data from that year.

### Fit The Model


In [ ]:
@model function gev_annmax(y)               
    μ ~ Normal(1000, 100)                   # <1>
    σ ~ truncated(Normal(0, 100); lower=0)  # <2>
    ξ ~ Normal(0, 0.5)                      # <3>

    y ~ GeneralizedExtremeValue(μ, σ, ξ)    # <4>
end

gev_model = gev_annmax(dat_annmax.residual) # <5>
n_chains = 4                                # <6>
n_per_chain = 5000                          # <7>
gev_chain = sample(gev_model, NUTS(), MCMCThreads(), n_per_chain, n_chains; drop_warmup=true)   # <8>
@show gev_chain

1. Location parameter prior: We know that this is roughly on the 1000 mm order of magnitude, but want to keep this relatively broad.
2. Scale parameter prior: This parameter must be positive, so we use a normal truncated at zero.
3. Shape parameter prior: These are usually small and are hard to constrain, so we will use a more informative prior.
4. The data is independently GEV-distributed as we've removed the long-term trend and are using long blocks.
5. Initialize the model.
6. We use multiple chains to help diagnose convergence.
7. This sets the number of iterations for each chain.
8. Sample from the posterior using NUTS and drop the iterations used to warmup the sampler.


In [ ]:
#| label: fig-gev-traceplot
#| fig-cap: Traceplots (left) and marginal distributions (right) from the MCMC sampler for the GEV model.
#| fig-alt: MCMC sampler output

plot(gev_chain)

From @fig-gev-traceplot, it looks like all of the chains have converged to the same distribution; the Gelman-Rubin diagnostic is also close to 1 for all parameters. Next, we can look at a corner plot to see how the parameters are correlated.


In [ ]:
#| label: fig-gev-corner
#| fig-cap: Corner plot for the GEV model.
#| fig-alt: GEV corner plot

corner(gev_chain)

@fig-gev-corner suggests that the location and scale parameters $\mu$ and $\sigma$ are positively correlated. This makes some intuitive sense, as increasing the location parameter shifts the bulk of the distribution in a positive direction, and the increasing scale parameter then increases the likelihood of lower values. However, if these parameters are increased, the shape parameter $\xi$ decreases, as the tail of the GEV  does not need to be as thick due to the increased proximity of outliers to the bulk.